# Instalamos librerias necesarias

In [ ]:
import requests

In [ ]:
url = "https://www.messivsronaldo.app/match-histories/ronaldo-match-history/" # ¡Cambia esto por la URL real!

try:
    response = requests.get(url)
    response.raise_for_status()  # Lanza una excepción para códigos de estado de error (4xx o 5xx)
    html_content = response.text
except requests.exceptions.RequestException as e:
    print(f"Error al obtener la página: {e}")
    html_content = None